<a href="https://colab.research.google.com/github/friverap/Data-Science-Core/blob/main/1.2TensorflowKerasActividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
tf.__version__

In [ ]:
keras.__version__


In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.head(15)

In [ ]:
df.info()

In [ ]:
# vamos a imputar los datos valtante, primero los asociadas a la variable Cabin, 
#voy a asumir que los valores faltantes se deben a que la persona no tenían una cabina asignada,
#además, codificaré en una variable booleana los que tenían cabina (1) y los que no (0)
df["CabinB"] = (df["Cabin"].notnull().astype('int'))
del df["Cabin"]

In [ ]:
df = df.fillna({"Embarked": "S"})

# Lo siguiente es estimar la edad de las personas, para esto simplemente voy a meter a las personas en determinados grupos( unkown, babies,etc.). Estas categorias me ayudarán a estimar una edad aproximada de las personas que no la tienen en el data set.


In [ ]:
df["Age"] = df["Age"].fillna(-1)
bins = [-2, 0, 5, 12, 18, 21, 35, 60, np.inf]
labels = ['Unknown', 'Baby', 'Child', 'Teenager',
          'post teen', 'Young Adult', 'Adult', 'Senior']
df['AgeGroup'] = pd.cut(df["Age"], bins, labels=labels)

In [ ]:
# Para estimar la edad de la clase unknown, me ayudaré de los titulos de las personas
df['title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
pd.crosstab(df['title'], df['Sex'])

In [ ]:
# reemplazar los titulos menos comunes
df['title_'] = df['title'].replace(['Capt', 'Col','Don', 'Dr', 'Major','Rev', 'Jonkheer'],'uncommon')

In [ ]:
pd.crosstab(df['title_'], df['Sex'])


In [ ]:
df['title_'] = df['title_'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
df['title_'] = df['title_'].replace('Mlle', 'Miss')
df['title_'] = df['title_'].replace('Ms', 'Miss')
df['title_'] = df['title_'].replace('Mme', 'Mrs')

In [ ]:
pd.crosstab(df['title_'], df['Sex'])

In [ ]:
df.head(15)

In [ ]:
# del la vista preliminar de arriba se puede ver que algunos titulos de mujeres no coinciden con su edad, entonces hay que arreglarlo
false_miss_index = df[(df['Sex'] == 'female') & (df['Age'] > 27) & (df['title_'] == 'Miss')]['title_'].index
df['title_'].iloc[false_miss_index] = df['title_'].iloc[false_miss_index].replace(['Miss'],'Mrs')

In [ ]:
false_mrs_index = df[(df['Sex'] == 'female') & (df['Age'] <= 21) & (df['title_'] == 'Mrs')]['title_'].index
df['title_'].iloc[false_mrs_index] = df['title_'].iloc[false_mrs_index].replace(['Mrs'],'Miss')

In [ ]:
#codificar los nuevos titulos
df['title_'] = df['title_'].map({"Mr": 1, "Miss": 2, "Mrs": 3,"Master": 4, "Royal": 5, "uncommon": 6})

In [ ]:
#Ahora se usará la información de los titulos para estimar la categoria 'Unknown'.
age_title_mapping = {1: "Young Adult", 2: "post teen",3: "Adult", 4: "Child", 5: "Adult", 6: "Adult"}
 
for i in range(df.shape[0]):
    if df["AgeGroup"][i] == "Unknown":
        df["AgeGroup"][i] = age_title_mapping[df["title_"][i]]


In [ ]:
df['AgeGroup'].unique()

In [ ]:
df.head(15)

In [ ]:
# ahora se puede eliminar las siguientes variables
df = df.drop(['Age','title'], axis=1)

In [ ]:
# Ahora se asignan valores numericos a las categorias de la edad
age_mapping = {'Baby': 1, 'Child': 2, 'Teenager': 3,
               'post teen': 4, 'Young Adult': 5, 'Adult': 6,
               'Senior': 7}
df['AgeGroup'] = df['AgeGroup'].map(age_mapping)
 
df.head()
 

In [ ]:
# eliminar la variable name y ticket pues no contienen más información relevante.
df = df.drop(['Name','Ticket'], axis=1)

In [ ]:
# vamos a ver la distribución de las clases de supervivientes 
f, ax = plt.subplots(1, 2, figsize=(12, 4))
df['Survived'].value_counts().plot.pie(
    explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=False)
ax[0].set_title('Sobrevivientes (1) Muertos (0)')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df, ax=ax[1])
ax[1].set_ylabel('Quantity')
ax[1].set_title('Sobrevivientes (1) Muertos (0)')
plt.show()

In [ ]:
#ahora por sexo
f, ax = plt.subplots(1, 2, figsize=(12, 4))
df[['Sex', 'Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survivors by sex')
sns.countplot('Sex', hue='Survived', data=df, ax=ax[1])
ax[1].set_ylabel('Quantity')
ax[1].set_title('Survived (1) and deceased (0): men and women')
plt.show()

In [ ]:
# vemos que se trata de un problema de calsificación donde su target está desbalanceado, para solventar esto se prodría implementar el método SMOTE. Pero no lo haré por ahora por falta de tiempo 

In [ ]:
#Ahora se asignaran valores numéricos a las categorias sex y embarks
sex_mapping = {"male": 0, "female": 1}
df['Sex'] = df['Sex'].map(sex_mapping)
 
embarked_mapping = {"S": 1, "C": 2, "Q": 3}
df['Embarked'] = df['Embarked'].map(embarked_mapping)
df.head()

In [ ]:
# por último vamos a clasificar la variable Fare values en cuatro grupos
df['FareBand'] = pd.qcut(df['Fare'], 4,labels=[1, 2, 3, 4])

df = df.drop(['Fare'], axis=1)
df.head()

In [ ]:
#Ahora a implementar los modelos base
from sklearn.model_selection import train_test_split
 
predictors = df.drop(['Survived', 'PassengerId'], axis=1)
target = df["Survived"]
x_train, x_val, y_train, y_val = train_test_split(
    predictors, target, test_size=0.2, random_state=0)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score
 
randomforest = RandomForestClassifier()
 
# Fit the training data along with its output
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
 
# Find the accuracy score of the model
acc_randomforest = round(precision_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

In [ ]:
# regresión logística
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
 
# Find the accuracy score of the model
acc_logreg = round(precision_score(y_pred, y_val) * 100, 2)
print(acc_logreg)

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

# Train the model using the training sets
knn.fit(x_train, y_train)

#Predict Output
y_pred= knn.predict(x_val) 
# Find the accuracy score of the model
acc_knn = round(precision_score(y_pred, y_val) * 100, 2)
print(acc_knn)

In [ ]:
#Red neuronal
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

krs = Sequential()
krs.add(Dense(128,activation="selu",input_dim=x_train.shape[1], kernel_initializer="ones"))
krs.add(Dense(128,activation="selu"))
krs.add(Dense(256,activation="selu"))
krs.add(Dense(1, activation="sigmoid"))

krs.compile(loss="binary_crossentropy",metrics=["accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()], optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))
krs.fit(x_train.values, y_train.values, epochs=500, verbose=0)

In [ ]:
#Predict Output
y_pred= krs.predict(x_val.values)

predicted_classes = np.where(y_pred >= 0.5, 1, 0)
# Find the accuracy score of the model
acc_keras = round(precision_score(predicted_classes, y_val) * 100, 2)
print(acc_keras)

In [ ]:
models = pd.DataFrame({'Model':['Random Forest','KNN','Logistic Regression','Keras'],'precision':[acc_randomforest,acc_knn,acc_logreg,acc_keras]})
models

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
sns.lineplot(x = "Model",
             y = "precision",
             data = models);

In [ ]:
# cosas por hacer:
# 1. Explorar espacio de hiperparametros de los modelos.
# 2. Validaciones cruzadas